# Generating malware detection features

## Getting the required disassembler

In [1]:
found = False
packages = !pip list
for package in packages:
    if "pefile" in package:
        found = True
        break
if not found:
    print("Package is missing, installing...")
    !pip install pefile
else:
    print("PEFile package found.")

PEFile package found.


## Collecting data about any application

### Checking for the PE file

In [2]:
from os.path import exists, expandvars
path = "$windir/notepad.exe"
exppath = expandvars(path)
print(exppath)
if exists(exppath):
    print("Notepad.exe is available for use.")
else:
    print("Use another Windows PE file.")

C:\WINDOWS/notepad.exe
Notepad.exe is available for use.


### Loading the Windows PE file

In [3]:
import pefile
exe_file = pefile.PE(exppath)

### Looking at the executable file sections

In [4]:
for section in exe_file.sections:
    print(section)

[IMAGE_SECTION_HEADER]
0x208      0x0   Name:                          .text
0x210      0x8   Misc:                          0x247FF   
0x210      0x8   Misc_PhysicalAddress:          0x247FF   
0x210      0x8   Misc_VirtualSize:              0x247FF   
0x214      0xC   VirtualAddress:                0x1000    
0x218      0x10  SizeOfRawData:                 0x24800   
0x21C      0x14  PointerToRawData:              0x400     
0x220      0x18  PointerToRelocations:          0x0       
0x224      0x1C  PointerToLinenumbers:          0x0       
0x228      0x20  NumberOfRelocations:           0x0       
0x22A      0x22  NumberOfLinenumbers:           0x0       
0x22C      0x24  Characteristics:               0x60000020
[IMAGE_SECTION_HEADER]
0x230      0x0   Name:                          .rdata
0x238      0x8   Misc:                          0x9280    
0x238      0x8   Misc_PhysicalAddress:          0x9280    
0x238      0x8   Misc_VirtualSize:              0x9280    
0x23C      0xC   Vi

In [5]:
for section in exe_file.sections:
    print(section.Name)

b'.text\x00\x00\x00'
b'.rdata\x00\x00'
b'.data\x00\x00\x00'
b'.pdata\x00\x00'
b'.didat\x00\x00'
b'.rsrc\x00\x00\x00'
b'.reloc\x00\x00'


### Examining the imported libraries

In [6]:
pefile.directory_entry_types

[('IMAGE_DIRECTORY_ENTRY_EXPORT', 0),
 ('IMAGE_DIRECTORY_ENTRY_IMPORT', 1),
 ('IMAGE_DIRECTORY_ENTRY_RESOURCE', 2),
 ('IMAGE_DIRECTORY_ENTRY_EXCEPTION', 3),
 ('IMAGE_DIRECTORY_ENTRY_SECURITY', 4),
 ('IMAGE_DIRECTORY_ENTRY_BASERELOC', 5),
 ('IMAGE_DIRECTORY_ENTRY_DEBUG', 6),
 ('IMAGE_DIRECTORY_ENTRY_COPYRIGHT', 7),
 ('IMAGE_DIRECTORY_ENTRY_GLOBALPTR', 8),
 ('IMAGE_DIRECTORY_ENTRY_TLS', 9),
 ('IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG', 10),
 ('IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT', 11),
 ('IMAGE_DIRECTORY_ENTRY_IAT', 12),
 ('IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT', 13),
 ('IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR', 14),
 ('IMAGE_DIRECTORY_ENTRY_RESERVED', 15)]

In [7]:
entries = []
for entry in exe_file.DIRECTORY_ENTRY_IMPORT:
    print (entry.dll)
    entries.append(entry)

b'KERNEL32.dll'
b'GDI32.dll'
b'USER32.dll'
b'api-ms-win-crt-string-l1-1-0.dll'
b'api-ms-win-crt-runtime-l1-1-0.dll'
b'api-ms-win-crt-private-l1-1-0.dll'
b'api-ms-win-core-com-l1-1-0.dll'
b'api-ms-win-core-shlwapi-legacy-l1-1-0.dll'
b'api-ms-win-shcore-obsolete-l1-1-0.dll'
b'api-ms-win-shcore-path-l1-1-0.dll'
b'api-ms-win-shcore-scaling-l1-1-1.dll'
b'api-ms-win-core-rtlsupport-l1-1-0.dll'
b'api-ms-win-core-errorhandling-l1-1-0.dll'
b'api-ms-win-core-processthreads-l1-1-0.dll'
b'api-ms-win-core-processthreads-l1-1-1.dll'
b'api-ms-win-core-profile-l1-1-0.dll'
b'api-ms-win-core-sysinfo-l1-1-0.dll'
b'api-ms-win-core-interlocked-l1-1-0.dll'
b'api-ms-win-core-libraryloader-l1-2-0.dll'
b'api-ms-win-core-winrt-string-l1-1-0.dll'
b'api-ms-win-core-synch-l1-1-0.dll'
b'api-ms-win-core-winrt-error-l1-1-0.dll'
b'api-ms-win-core-string-l1-1-0.dll'
b'api-ms-win-core-winrt-l1-1-0.dll'
b'api-ms-win-core-winrt-error-l1-1-1.dll'
b'api-ms-win-eventing-provider-l1-1-0.dll'
b'api-ms-win-core-synch-l1-2-0.dll

In [8]:
print(entries[0].dll)
for function in entries[0].imports:
    print(f"\t{function.name}")

b'KERNEL32.dll'
	b'GetProcAddress'
	b'CreateMutexExW'
	b'AcquireSRWLockShared'
	b'DeleteCriticalSection'
	b'GetCurrentProcessId'
	b'GetProcessHeap'
	b'GetModuleHandleW'
	b'DebugBreak'
	b'IsDebuggerPresent'
	b'GlobalFree'
	b'GetLocaleInfoW'
	b'CreateFileW'
	b'ReadFile'
	b'MulDiv'
	b'GetCurrentProcess'
	b'GetCommandLineW'
	b'HeapSetInformation'
	b'FreeLibrary'
	b'LocalFree'
	b'LocalAlloc'
	b'FindFirstFileW'
	b'FindClose'
	b'FoldStringW'
	b'GetModuleFileNameW'
	b'GetUserDefaultUILanguage'
	b'GetLocalTime'
	b'HeapFree'
	b'HeapAlloc'
	b'WideCharToMultiByte'
	b'WriteFile'
	b'GetFileAttributesW'
	b'LocalLock'
	b'GetACP'
	b'LocalUnlock'
	b'DeleteFileW'
	b'SetEndOfFile'
	b'GetFileAttributesExW'
	b'GetFileInformationByHandle'
	b'CreateFileMappingW'
	b'MapViewOfFile'
	b'MultiByteToWideChar'
	b'LocalReAlloc'
	b'UnmapViewOfFile'
	b'GetFullPathNameW'
	b'LocalSize'
	b'GetStartupInfoW'
	b'lstrcmpiW'
	b'FindNLSString'
	b'GlobalLock'
	b'GlobalUnlock'
	b'GlobalAlloc'
	b'GetDiskFreeSpaceExW'
	b'CreateDire

## Extracting strings from an executable

In [9]:
!strings -n 10 %windir%\Notepad.exe


Strings v2.54 - Search for ANSI and Unicode strings in binary images.
Copyright (C) 1999-2021 Mark Russinovich
Sysinternals - www.sysinternals.com

!This program cannot be run in DOS mode.
\$ UVWAVAWH
WATAUAVAWH
l$ VWATAVAWH
WATAUAVAWH
 A_A^A]A\_
WATAUAVAWH
 A_A^A]A\_
UVWATAUAVAWH
F0D8#ubD8c
`A_A^A]A\_^]
WATAUAVAWH
UATAUAVAWH
UVWATAUAVAWH
A_A^A]A\_^]
@SUVWATAVAWH
PA_A^A\_^][
WATAUAVAWH
L$ UVWATAUAVAWH
pA_A^A]A\_^]
UVWATAUAVAWH
A_A^A]A\_^]
UVWATAUAVAWH
A_A^A]A\_^]
[ UVWAVAWH
|$ UATAUAVAWH
t$ UWAUAVAWH
H!_(H!_0H!_8H!_@3
t$ UWAUAVAWH
H!\$0H!\$8D
l$ VWATAVAWH
UVWATAUAVAWH
A_A^A]A\_^]
@USVWATAVAWH
A_A^A\_^[]
@USVWATAVAWH
PA_A^A\_^[]
D$0H9D$(utH
WATAUAVAWH
x UATAUAVAWH
WATAUAVAWH
D$0H9D$(utH
UVWATAUAVAWH
A_A^A]A\_^]
WATAUAVAWH
 A_A^A]A\_
@USVWATAUAVAWH
A_A^A]A\_^[]
UVWATAUAVAWH
@A_A^A]A\_^]
L$H9L$@v\H
x UATAUAVAWH
x UATAUAVAWH
UVWATAUAVAWH
@A_A^A]A\_^]
D$p+D$h9D$xuoL
@USVWATAVAWH
pA_A^A\_^[]
USVWATAUAVAWH
A_A^A]A\_^[]
L$ USVWATAUAVAWH
HA_A^A]A\_^[]
USVWATAVAWH
`A_A^A\_^[]
UATAUAVAWH
ADVAPI3

In [10]:
exe_strings = !strings -n 10 %windir%\Notepad.exe
for exe_string in exe_strings:
    if "%" in exe_string:
        print(exe_string)

%hs(%u)\%hs!%p: 
(caller: %p) 
%hs(%d) tid(%x) %08X %ws
Msg:[%ws] 
CallContext:[%hs] 
[%hs(%hs)]
Local\SM0:%d:%d:%hs
%s%c*.txt%c%s%c*.*%c
%s\%s.autosave
%08lX-%04X-%04x-%02X%02X-%02X%02X%02X%02X%02X%02X
